<a href="https://colab.research.google.com/github/MONIK-HUB/BERT/blob/main/cs18b1037bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install praw

     |████████████████████████████████| 1.3MB 11.5MB/s 
     |████████████████████████████████| 890kB 17.2MB/s 
     |████████████████████████████████| 2.9MB 52.5MB/s 
     |████████████████████████████████| 1.1MB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=97ea1b638c430f54119001807d079ae476def5f673fe9216855cbc1ff9f488e3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 153kB 12.6MB/s 
     |████████████████████████████████| 204kB 12.8MB/s 


In [ ]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer 
from matplotlib import pyplot as plt 
import tensorflow as tf
import praw 
import pandas as pd 

In [ ]:
model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing TFBertForTokenClassification: ['dropout_147']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english and are newly initialized: ['dropout_73']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label_list = [
    "O",       
      "B-MISC",  
    "I-MISC",  
    "B-PER",   
    "I-PER",  
    "B-ORG",   
    "I-ORG",   
    "B-LOC",   
    "I-LOC"    
]

In [ ]:
reddit = praw.Reddit(client_id='c1QVneewT5wHSA', 
                     client_secret='aOdi_F9KDS7W92jFz9JCl-zcN4CknA',
                     user_agent='cse project')

In [ ]:

def replies_of(top_level_comment, comment_list):
    if len(top_level_comment.replies) == 0:
        return
    else:
        for num, comment in enumerate(top_level_comment.replies):
            try:
                comment_list.append(str(comment.body))
            except:
                continue
            replies_of(comment, comment_list)

In [ ]:
count = 0
master_dict = {'I-LOC': [], 'I-ORG': [], 'I-PER': [], 'B-LOC': [], 'B-ORG': [], 'B-PER': []} 
word_temp = ''
current_tag = ''
old_tag = ''
print_dict = {}

In [ ]:

list_of_subreddit = ['india']

In [15]:
for j in list_of_subreddit:
    top_posts = reddit.subreddit(j).top('week', limit=1)
    comment_list = [] 
    for submission in top_posts:
        submission_comm = reddit.submission(id=submission.id)
        comment_list.append(str(submission.title))

        for count, top_level_comment in enumerate(submission_comm.comments):
            try:
                replies_of(top_level_comment, comment_list)
            except:
                continue

In [16]:
for sequence in comment_list:
    if len(sequence) > 512: 
        continue
    
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
    inputs = tokenizer.encode(sequence, return_tensors="tf")
    outputs = model(inputs)[0]
    predictions = tf.argmax(outputs, axis=2)
    
    list_bert = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]
    for i in list_bert:
        if i[1] in ['O', 'B-MISC', 'I-MISC']:
            
            if len(current_tag) > 0:
                without_space_word = word_temp.strip()
                if len(without_space_word) > 1:
                    master_dict[current_tag].append(without_space_word)
            count = 0
            word_temp = ''
            current_tag = ''
            continue
        else:
            current_tag = i[1]

            if old_tag != current_tag and len(old_tag) > 0:
                without_space_word = word_temp.strip()
                if len(without_space_word) > 1:
                    master_dict[old_tag].append(without_space_word)
                count = 0
                word_temp = ''
                current_tag = ''

            if i[0].startswith('##'): 
                word_temp += i[0][2:].upper()
            elif i[1] in ['I-PER', 'I-ORG', 'I-LOC', 'B-LOC', 'B-ORG', 'B-PER']:
                word_temp += " " + i[0].upper()
                current_tag = i[1]
                count += 1
            old_tag = current_tag
            


In [17]:
print(master_dict)


{'I-LOC': ['HYDERABAD', 'INDIA', 'INDIA', 'INDIA', 'INDIA', 'HYDERABAD', 'INDIA', 'HITECH CITY', 'SECUNDERABAD', 'INDIA', 'DIA', 'AMERICA', 'US', 'ERIC', 'USA', 'CANADA', 'EUROPE', 'US', 'INDIA', 'USA', 'INDIA', 'AMERICA', 'AMERICA', 'INDIA', 'INDIA', 'SOUTH AMERICA', 'TAMBARAM', 'INDIA', 'HYDERABAD', 'YDERABAD', 'MUMBAI', 'PUNE', 'YDERABAD', 'KALYAN', 'BANGALORE', 'DIA', 'HYDERABAD', 'INDIA', 'INDIA', 'INDIA', 'INDIA', 'HYDERABAD', 'INDIA', 'HITECH CITY', 'SECUNDERABAD', 'INDIA', 'DIA', 'AMERICA', 'US', 'ERIC', 'USA', 'CANADA', 'EUROPE', 'US', 'INDIA', 'HYDERABAD', 'INDIA', 'INDIA', 'INDIA', 'INDIA', 'HYDERABAD', 'INDIA', 'HITECH CITY', 'SECUNDERABAD', 'INDIA', 'DIA', 'AMERICA', 'US', 'ERIC', 'USA', 'CANADA', 'EUROPE', 'US', 'INDIA', 'USA', 'INDIA', 'AMERICA', 'AMERICA', 'INDIA', 'INDIA', 'SOUTH AMERICA', 'TAMBARAM', 'INDIA', 'HYDERABAD', 'YDERABAD', 'MUMBAI', 'PUNE', 'YDERABAD', 'KALYAN', 'BANGALORE', 'DIA', 'BANG'], 'I-ORG': ['ACTIVA / ACCESS', 'PRS LEGISLATIVE', 'ACAB', 'ACAB', 'EM